# Final Data setup and Modeling

In [1]:
# imports
import numpy as np
import tensorflow as tf

2023-03-21 08:21:10.633199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load data
Xtrain = np.load("../data/train/Xtrain.npy")
Ytrain = np.load("../data/train/Ytrain.npy")
Xtest = np.load("../data/test/Xtest.npy")
Ytest = np.load("../data/test/Ytest.npy")

In [3]:
# balance classes
Ytrain_use = Ytrain[:1124]
Xtrain_use = Xtrain[:1124]
Ytest_use = Ytest[:282]
Xtest_use = Xtest[:282]

## convert grayscale image to rgb images to make compatible with TF

In [4]:
# define image reshaper function to create rgb values for each pixel in the image
def img_reshaper(orig_img):
    new_img = []
    for row in orig_img:
        nrow = []
        for item in row:
            nrow.append([item, item, item])
        new_img.append(nrow)
    return new_img

In [ ]:
# reshape Xtrain
Xtrain_temp = []
c = 1
for img in Xtrain_use:
    Xtrain_temp.append(img_reshaper(img))
    print(f"{c} out of 1124 complete.")
    c+=1
Xtrain_new = np.array(Xtrain_temp)

1 out of 1124 complete.
2 out of 1124 complete.
3 out of 1124 complete.
4 out of 1124 complete.
5 out of 1124 complete.
6 out of 1124 complete.
7 out of 1124 complete.
8 out of 1124 complete.
9 out of 1124 complete.
10 out of 1124 complete.
11 out of 1124 complete.
12 out of 1124 complete.
13 out of 1124 complete.
14 out of 1124 complete.
15 out of 1124 complete.
16 out of 1124 complete.
17 out of 1124 complete.
18 out of 1124 complete.
19 out of 1124 complete.
20 out of 1124 complete.
21 out of 1124 complete.
22 out of 1124 complete.
23 out of 1124 complete.
24 out of 1124 complete.
25 out of 1124 complete.
26 out of 1124 complete.
27 out of 1124 complete.
28 out of 1124 complete.
29 out of 1124 complete.
30 out of 1124 complete.
31 out of 1124 complete.
32 out of 1124 complete.
33 out of 1124 complete.
34 out of 1124 complete.
35 out of 1124 complete.
36 out of 1124 complete.
37 out of 1124 complete.
38 out of 1124 complete.
39 out of 1124 complete.
40 out of 1124 complete.
41 out of

In [ ]:
# check shape for correctness
Xtrain_new.shape

In [ ]:
# save modified Xtrain and Ytrain
np.save("../data/train/Xtrain_final.npy", Xtrain_new)
np.save("../data/train/Ytrain_final.npy", Ytrain_use)

In [ ]:
# reshape Xtest
Xtest_temp = []
c = 1
for img in Xtest_use:
    Xtest_temp.append(img_reshaper(img))
    print(f"{c} out of 1124 complete.")
    c+=1
Xtest_new = np.array(Xtrain_temp)

In [ ]:
# check shape for correctness
Xtest_new.shape

In [ ]:
# save modified Xtest and Ytest
np.save("../data/train/Xtest_final.npy", Xtest_new)
np.save("../data/train/Ytest_final.npy", Ytest_use)

## modeling

In [4]:
# setup CNN using Inception-v3 (using google JAMA paper for this choice)
base_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights=None,
    input_shape=(Xtrain_use.shape[1],Xtrain_use.shape[2],1)
)

#base_model = PTModel(input_shape =  t_x.shape[1:], include_top = False, weights = 'imagenet')

2023-03-21 08:21:31.861689: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# unfreeze the base model
base_model.trainable = True

In [7]:
# create new model that we are going to iterate on
inputs = tf.keras.Input(shape=(1776, 2368))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

In [8]:
# train!
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.Accuracy()])
model.fit(x=Xtrain_use.reshape((Xtrain_use.shape[0],Xtrain_use.shape[1],Xtrain_use.shape[2],1)), y=Ytrain_use, epochs=20, validation_data=(Xtest_use.reshape((Xtest_use.shape[0],Xtrain_use.shape[1],Xtrain_use.shape[2],1)), Ytest_use))

Epoch 1/20


: 

: 

## modeling 2

In [7]:
#from keras.applications.vgg16 import VGG16 as PTModel
#from keras.applications.inception_resnet_v2 import InceptionResNetV2 as PTModel
from keras.applications.inception_v3 import InceptionV3 as PTModel
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda
from keras.models import Model
in_lay = Input((Xtrain_use.shape[1],Xtrain_use.shape[2],1))
base_pretrained_model = PTModel(input_shape =  (Xtrain_use.shape[1],Xtrain_use.shape[2],1), include_top = False, weights = None)
base_pretrained_model.trainable = False
pt_depth = base_pretrained_model.get_output_shape_at(0)[-1]
pt_features = base_pretrained_model(in_lay)
from keras.layers import BatchNormalization
bn_features = BatchNormalization()(pt_features)

# here we do an attention mechanism to turn pixels in the GAP on an off

attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(Dropout(0.5)(bn_features))
attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
attn_layer = Conv2D(8, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
attn_layer = Conv2D(1, 
                    kernel_size = (1,1), 
                    padding = 'valid', 
                    activation = 'sigmoid')(attn_layer)
# fan it out to all of the channels
up_c2_w = np.ones((1, 1, 1, pt_depth))
up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
               activation = 'linear', use_bias = False, weights = [up_c2_w])
up_c2.trainable = False
attn_layer = up_c2(attn_layer)

mask_features = multiply([attn_layer, bn_features])
gap_features = GlobalAveragePooling2D()(mask_features)
gap_mask = GlobalAveragePooling2D()(attn_layer)
# to account for missing values from the attention model
gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
gap_dr = Dropout(0.25)(gap)
dr_steps = Dropout(0.25)(Dense(128, activation = 'relu')(gap_dr))
out_layer = Dense(t_y.shape[-1], activation = 'softmax')(dr_steps)
retina_model = Model(inputs = [in_lay], outputs = [out_layer])
from keras.metrics import top_k_categorical_accuracy
def top_2_accuracy(in_gt, in_pred):
    return top_k_categorical_accuracy(in_gt, in_pred, k=2)

retina_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                           metrics = ['categorical_accuracy', top_2_accuracy])
retina_model.summary()

2023-03-20 19:32:13.119314: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: The layer inception_v3 has never been called and thus has no defined output shape.